In [1]:
import json
import pandas as pd

In [116]:
with open("data/db") as file:
    db = file.readlines()

db = [json.loads(line.removesuffix("\n")) for line in db]

In [117]:
def convert_row(row):
    return {elem["trait_type"]: elem["value"] for elem in row}

In [118]:
df_db = pd.DataFrame.from_dict(db)
df_exploded = pd.concat([df_db, df_db["metadata"].apply(pd.Series)], axis=1)

In [119]:
attributes = df_exploded["attributes"].apply(convert_row)
attributes = pd.DataFrame.from_dict(attributes.tolist())

In [120]:
df_final = pd.concat([df_exploded, attributes], axis=1)
df_final["image"] = df_final["image"].str.removeprefix("ipfs://")
df_final = df_final.loc[:, ['image',  'Mouth', 'Background', 'Hat', 'Eyes',
       'Clothes', 'Fur', 'Earring']]

In [121]:
df_final.to_csv("apes_info.csv", index=False)

In [124]:
df_final["Mouth"].nunique()

33

In [131]:
df_final["Fur"].value_counts()

Fur
Brown           1370
Dark Brown      1352
Black           1229
Golden Brown     778
Cream            636
Tan              626
Pink             511
Gray             496
Blue             490
Red              474
Cheetah          406
White            397
Zombie           302
Robot            265
Dmt              215
Death Bot        175
Noise            155
Trippy            77
Solid Gold        46
Name: count, dtype: int64

In [125]:
df_final["Background"].nunique(), df_final["Hat"].nunique(), df_final["Eyes"].nunique()

(8, 36, 23)

In [126]:
df_final["Clothes"].nunique(), df_final["Fur"].nunique(), df_final["Earring"].nunique()

(43, 19, 6)